# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv ('/datasets/data.csv')
data.info()
print(data.head(10))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4  

**Вывод**

1.1 По начальной информации видно, что в столбцах дней стажа и ежемесячного дохода имеются пропуски.

1.2 В столбце с видом образования клиента некоторые значения введены заглавными буквами.

1.3 В столбцах цели кредита и семейного положения одинаковый смысл выражен разными словосочетаниями.

1.4 В столбце стажа наблюдаются отрицательные и не целочисленные значения.

## Шаг 2. Предобработка данных

### Обработка пропусков

2.1.1 Узнаем количество пропущенных значений в данных. 

In [2]:
# воспользуемся методом isna() и sum()
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


2.1.2 Обнаружено 2170 пропущенных значений в столбцах с ежемесячным доходом и трудовым стажем.

2.1.2 Посмторим, что за строчки с пропущенными значениями в столбцах c ежемесячным доходом и трудовым стажем.

In [3]:
# отсортируем датафрейм по столбцу с трудовым стажем.
data.sort_values('days_employed').tail(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21369,2,NaN,42,среднее,1,в разводе,3,M,компаньон,0,NaN,покупка жилой недвижимости
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21391,0,NaN,52,среднее,1,женат / замужем,0,F,компаньон,0,NaN,покупка жилья для семьи
21407,1,NaN,36,среднее,1,женат / замужем,0,F,компаньон,0,NaN,строительство жилой недвижимости
21414,0,NaN,65,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,покупка своего жилья
21415,0,NaN,54,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с жильем
21423,0,NaN,63,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,сделка с автомобилем
21426,0,NaN,49,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,недвижимость
21432,1,NaN,38,неоконченное высшее,2,Не женат / не замужем,4,F,сотрудник,0,NaN,операции с жильем
21463,1,NaN,35,высшее,0,гражданский брак,1,M,сотрудник,0,NaN,на проведение свадьбы


2.1.3 Как видно из среза таблицы, то никакой закономерности у клиентов с незаполнеными данными о доходе и стаже нету. 2174 строчки с пропущенными значениями данными это чуть больше 10% выборки клиентов. Удаление этих строк - это упущение потенциальных клиентов для банка.

2.1.4 Предлагаю воспользоваться средним  для стажа работы и медианой для ежемесячной прибыли , но только не по всей выборке клиентов, а по возрастным катеогриям 18-29, 30 - 39, 40 - 49, 50 - 59, 60+. Для этого создадим функцию для сортировки клиентов по возрастным катеогриям age_group.

In [4]:
# узнаем минимальный возраст клиентов
print ('Минимальный возраст клиента:', data['dob_years'].min(), 'лет.')
print ('Максимальный возраст клиента:', data['dob_years'].max(), 'лет.')

Минимальный возраст клиента: 0 лет.
Максимальный возраст клиента: 75 лет.


2.1.5 Неожиданно выяснилось, что у нас есть клиенты, которые ещё не родились. узнаем сколько же таких клиентов в нашей выборке.

In [5]:
# находим все строки с условием, что в ячейке столбца с возрастом - 0 лет
print('Количество клиентов с нулевым возрастом: ', data.loc[data['dob_years'] == 0]['dob_years'].count())

Количество клиентов с нулевым возрастом:  101


2.1.6 У 101 клиента в выборке возраст указан как 0 и из 101.

2.1.7 Cортируем клиентов согласно пункту 2.1.4, 101 клиент без возраста ставим "возраст не указан". В дальнейшем при анализе датафрейма они учитываться не будут.

In [6]:
# напишем функцию для сортировки клиентов по возрастным группам:
def age_groups(dob_years):
    """
    Возвращает возрастную группу по значению возраста dob_years и пола gender, со следующими правилами:
    - "18-29 лет" при 18 <= dob_years <= 29
    - "30-39 лет" при 30 <= dob_years <= 39
    - "40-49 лет" при 40 <= dob_years <= 49 
    - "50-59 лет" при 50 <= dob_years <= 59
    - "60+ лет" dob_years >= 60
    - остальные 'Возраст не указан'
    """
   
    
    if 18 <= dob_years <= 29:
        return '18-29 лет'     
          
    if 30 <= dob_years <= 39:
        return '30-39 лет'
       
    if 40 <= dob_years <= 49:
        return '40-49 лет'
       
    if 50 <= dob_years <= 59:
        return '50-59 лет'
           
    if 60 <= dob_years:
        return '60+ лет'
        
    return 'Возраст не указан'

#применим нашу функцию к столбцу с 'dob_years' а результаты сортировки укажем в новом столбце age_group

data['age_group'] = data['dob_years'].apply(age_groups)

# проверяем результат работы функции
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,40-49 лет
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,30-39 лет
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,30-39 лет
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,30-39 лет
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,50-59 лет
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,18-29 лет
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,40-49 лет
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,50-59 лет
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,30-39 лет
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,40-49 лет


2.1.8 Теперь у нас есть новый столбце в таблице с отображением возрастной группы каждого клиента. Для дальнешей работы со стажем и ежемесячным доходом заменим все пропущенные значения на "0"



In [7]:
# заменим все пропущенные значения на "0"
data = data.fillna(0)

# выполним проверку
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

**Вывод**

2.1.7 Обнаружено 2170 клиентов с неуказанным трудовым стажем и ежемесячным доходом, что составляет порядка 10% общей выборки. Два этих параметра связаны, так что причиной пропуска этих значений может быть утеря данных, или возможно смена работы клиента, без предоставления актульного места работы. При потенцильном обращении этого клиента с целью получения кредита необходимо восстановить эти данные.

2.1.8 Предлагается заменить отсутствующие значения с помощью медианы и среднего значения соглсано пп. 2.1.4, чтобы сохранить 10%  общей выборки клиентов. Так как в данной работе необходимо оценить платежеспособность клиентов от количества детей и семейного положения, считаю что усреднение  пропущенных значений по выбранным мной возрастным катеоориям не критично скажется на поставленной задаче и сохранит нам клиентов из всей выборки.

2.1.9 Для устранение пропусков в столбцах с ежемесячным доходом и стажем была применена категоризация по возрасту всех клиентов. А все значения с пропусками заменены на "0" для дальнейшей работы с данными из этих столбцов. 

Для дальнейшего расчета медианы ежемесячного дохода и среднего стажа для каждой возрастной группы. Замена пропусков на эти значения будет выполнена в пунтке 2.5. Так перед этим необходимо выполнить замену типа данных в столбце со стажем работы, который относится к пункту 2.2. А также в пунтке 2.3 устранить дубликаты.

2.1.10 Клиенты с "нулевым" возрастом составляют 101 - это  0,5% всей выборки и будут добавлены в категорию "Возраст не указан" и не будут учитываться.

### Замена типа данных

2.2.1 В столбце со стажем работы указаны дни с отрицательными и с положительными значениями, считаю это ошибкой ввода данных, и предлагаю взять модуль у отрицательных значений и преобразовать в целочисленные значения, так как дни как правило указываются целиком.

In [8]:
#переведем дни в целочисленные значения.

data['days_employed'] = data['days_employed'].astype('int')

# применим метод abs() к столбцу 'days_employed' для взятия чисел по модулю и проверим

data['days_employed'] = data['days_employed'].abs()
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,40-49 лет
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,30-39 лет
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,30-39 лет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,30-39 лет
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,50-59 лет


2.2.2 Теперь в столбце со стажем указанны целые дни, без всяких отрицательных значений.

2.2.3 В столбце с доходом уменьшим количетсво знаков после запятой до 2. Считаю такую точность тоже излишней.


In [9]:
# применим метод round() для уменьшения количества знаков после запятой до 2 и проверим.

data['total_income'] = data['total_income'].round(decimals = 2)

data['total_income']

0        253875.64
1        112080.01
2        145885.95
3        267628.55
4        158616.08
           ...    
21520    224791.86
21521    155999.81
21522     89672.56
21523    244093.05
21524     82047.42
Name: total_income, Length: 21525, dtype: float64

2.2.4 Проверим какое количество детей у наших клиентов.

In [10]:
# используем метод  value_counts() для столбца с детьми
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

2.2.5 Обнаружилось 47 человек с "-1" ребенком в семье. Исправим эту ошибку с помощью вычисления модуля, аналогично как с со столбцом стажа работы.

2.2.6 Так же клиенты с 20 детьми - явная ошибка в датафрейме, взглянем на этих клиентов поподбронее.

In [11]:
# применим функцию модуля к столбцу 'children' и проверим

data['children'] = data['children'].abs()

data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [12]:
# выведем данные о клиентах в с детьми больше 20
data[data['children'] == 20].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.87,покупка жилья,18-29 лет
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.74,покупка недвижимости,40-49 лет
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.54,получение образования,50-59 лет
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.84,операции с коммерческой недвижимостью,50-59 лет
2941,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.94,на покупку автомобиля,Возраст не указан
3302,20,0,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,0.00,профильное образование,30-39 лет
3396,20,0,56,высшее,0,женат / замужем,0,F,компаньон,0,0.00,высшее образование,50-59 лет
3671,20,913,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.49,на покупку подержанного автомобиля,18-29 лет
3697,20,2907,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.69,на покупку подержанного автомобиля,40-49 лет
3735,20,805,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.65,ремонт жилью,18-29 лет


2.2.7 Клиенты с 20 детьми - это явная ошибка, так как в остальных параметрах не прослеживается никакой зависимости.  Возможно тут лишний "0". Возможная причина - сбои при передачи данных или ошибка при начальном вводе данных о клиенте. Считаю, что эти данные не стоит учитывать.

**Вывод**

2.2.8 В столбце со стажем работы и количеством детей были обнаружены отрицательный значения. Считаю это возможной ошибкой в данных. Сначала данные стажа работы были преобразованы в целочисленный тип данных, все таки дни принято считать целиком. Далее был применен модуль для всех отрицательных значений в столбце стажа работы и количества детей.

2.2.9 В столбце с количеством детей обнаружены клиенты с 20 детьми, считаю это ошибкой в данных и при анализе связанным с количеством детей эти клиенты учитываться не будут.

2.2.10 В столбце с ежемесячной прибылью было сокращено количество знаков после запятой до 2. Вроде меньше как 1 копейка у нас номинала нету.

### Обработка дубликатов

2.3.1 Проверим наличие дубликатов в датафрейме. Если найдутся удаляем, и проверяем результат.

In [13]:
# получение количество дубликатов в датафрейме
data.duplicated().sum()

54

In [14]:
# удаление всех дубликатом методом drop_duplicates()
data = data.drop_duplicates().reset_index(drop = True)

In [15]:
# проверка на удаление
data.duplicated().sum()

0

2.3.2 Обнаружено 54 дубликата. Дубликаты могли появиться в следствии сбоя передачи или записи данных. Стоит обратить внимание и разобраться с появлением такого "мусора".

2.3.3 Посмотрим на уникальные значения в столбцах с типом занятости и семейным положением, возможно один и тот же статус выражен одинаково, а также уровень образования клиента.

In [16]:
# воспользуемся методом unique() и посмторим все уникальные значение в столбе с типом занятости на род неявных дубликатов
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [17]:
# тоже самое для столбца с семейным положением
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [18]:
# аналогично для стобца с уровнем образования
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

2.3.5 В столбце с типом занятости и семейным положением неявных дубликатов  не выявлено.

2.3.6 А вот в столбце с видом образования обнаружены дубликаты, которые уберутся приведением к нижнему регистру.

In [19]:
# применим метод lower() для того, чтобы привести все значения в столбце 'education' к нижнему регистру

data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

2.3.7 После приведения к нижнему регистру столбца с уровнем образования ещё раз проверим датафрейм на наличие дубликатов.

In [20]:
# получение количество дубликатов в датафрейме
data.duplicated().sum()

17

2.3.8 Обнаружилось 17 новых дубликатов, убираем их из датафрейма.

In [21]:
# удаление всех дубликатом методом drop_duplicates()
data = data.drop_duplicates().reset_index(drop = True)

#проверяем наличие дубликатов после удаления
data.duplicated().sum()

0

2.3.9 Дубликаты в столбце с уровнем образования успешно убраны.

2.3.10 Посмторим, наличие неявных дубликатов в стобце с целью кредита и пола клиента.

In [22]:
# воспользуемся методом value_counts() для столбца 'gender'
data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

In [23]:
# воспользуемся методом value_counts() для столбца 'purpose'
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

2.3.11 В стоблце с целью кредита наблюдается множество строк, где цель кредита одинаковая, но записана разными словами. Данный вид дубликатов необходимо решать в следующем пункте с работы с помощью лемматизации.

2.3.12 Обнаружен артефакт датафрейма - 1 человек с неопределнным полом. Посмоторим на этого клиента.

In [24]:
# вызовем строку по условию значения пола в столбце "gender"
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
10684,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.16,покупка недвижимости,18-29 лет


2.3.13 Так как это всего 1 человек, то его можно не учитывать в общей выборке.

**Вывод**

2.3.14 Был применён стандартный метод поиска дубликатов во всем датафрейме и подсчитано количество. Обнаружено и удалено 54 дубликата. Причина появления может быть некорректная обработка или передачи данных.

2.3.15 Использовался ручной поиск дубликатов в столбцах с текстовыми данными ( вид занятости, семейное положение, пол клиента, образование). В столбце с образованием обнаружены, что одинаковый уровня образования записывался прописными и строчными буквами.
Эта ошибка была устранена. Далее после приведения уровня образования к нижнему регистру датафрейм был снова проверен на наличие дубликатов, и было обнаружено и удалено ещё 17 дубликатов.

Так же обнаружился 1 клиент с неопределенным полом, принято решение не учитывать его в общей выборке.

2.3.16 В столбце с целью кредита необходимо применить лемматизацию для устранения смысловых дубликатов для одинаковой цели  кредитования.

### Лемматизация

2.4.1 Лемматизируем значения столбца с целью кредита для устранения смысловых дубликатов.

In [25]:
# импортируем pymystem3
from pymystem3 import Mystem
m = Mystem()


# функция для лемматизации значений в столбце 'purpose'
def purpose_lemmas(purpose):
    lemmas = m.lemmatize(purpose)
    return lemmas

# применяем лемматизацию к столбцу 'purpose'
data['purpose']=data['purpose'].apply(purpose_lemmas)


In [26]:
# функция для выбора из списка лемматизированных слов нпужного по смыслу и сохранению в таблице:
"""
    Функция из столбца 'purpose' берет лемматизированный и перебряирает его пока не найдет ключевое слова в каждом ряду столбца.
    Как только ключевое слово найдено, она возвращает его в ячейку столбца.
    Все ряды с ключевым словом "жилье" возвращают "недвижимость".
""" 
def final_purpose(row):
    for word in row:
        if 'автомобиль'in row:
            return 'автомобиль'
        elif 'свадьба'in row:
            return 'свадьба'
        elif 'недвижимость' in row:
            return 'недвижимость'
        elif 'образование' in row:
            return 'образование'
        return 'недвижимость'
    
           

data['purpose'] = data['purpose'].apply(final_purpose)


data['purpose'].value_counts()       
     

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

**Вывод**

Рассмотрены изначальные цели кредита клиентов банка. Из множество вариантов было выделено 4 основных направления: недвижимость, свадьба, образование и автомобиль. С помощью лемматизации было убрано множество смысловых дубликатов целей кредитования и выполнена сортировка клиентов по 4 основым категориям.

### Категоризация данных

2.5.1 Вернемся к созданию функции для сортировки клиентов по возрастным категориям. Расчитаем медиану ежемесячной прибыли и средний стаж для каждой возрастной группы и заменим "0" на эти значения для каждой возрастной группы соотвесвтенно.

2.5.2 Расчитываем все необходимые медианы и средние для каждой возрастной группы.

In [27]:

# медиана дохода и средний стаж для категории 18-29
median_income_20 = data[data['age_group'] == '18-29 лет']['total_income'].median()
mean_employed_20 = data[data['age_group'] == '18-29 лет']['days_employed'].mean()
mean_employed_20 = mean_employed_20.astype('int')

# медиана дохода и средний стаж для категории 30-39
median_income_30 = data[data['age_group'] == '30-39 лет']['total_income'].median()
mean_employed_30 = data[data['age_group'] == '30-39 лет']['days_employed'].mean()
mean_employed_30 = mean_employed_30.astype('int')

# медиана дохода и средний стаж для категории 40-49
median_income_40 = data[data['age_group'] == '40-49 лет']['total_income'].median()
mean_employed_40 = data[data['age_group'] == '40-49 лет']['days_employed'].mean()
mean_employed_40 = mean_employed_40.astype('int')

# медиана дохода и средний стаж для категории 50-59
median_income_50 = data[data['age_group'] == '50-59 лет']['total_income'].median()
mean_employed_50 = data[data['age_group'] == '50-59 лет']['days_employed'].mean()
mean_employed_50 = mean_employed_50.astype('int')

# медиана дохода и средний стаж для категории 60+
median_income_60 = data[data['age_group'] == '60+ лет']['total_income'].median()
mean_employed_60 = data[data['age_group'] == '60+ лет']['days_employed'].mean()
mean_employed_60 = mean_employed_60.astype('int')

2.5.3 Заменим "0" значения в столбцах со стажем и ежемесячным доходом на рассчитанные значения медианы и среднего.

In [28]:
# заменяем "0" в столбцах 'total_income' и 'days_employed' на рассчитынаю медиану и среднюю для возрастной группы 18-29 лет.
data.loc[(data['age_group'] == '18-29 лет') & (data['total_income'] == 0), 'total_income'] = median_income_20
data.loc[(data['age_group'] == '18-29 лет') & (data['days_employed'] == 0), 'days_employed'] = mean_employed_20

# проверяем наличие "0" столбцах 'total_income' и 'days_employed' для возрастной группы 18-29 лет.
print (data[data['age_group'] == '18-29 лет']['total_income'].min())
print (data[data['age_group'] == '18-29 лет']['days_employed'].min())

# заменяем "0" в столбцах 'total_income' и 'days_employed' на рассчитынаю медиану и среднюю для возрастной группы 30-39 лет.
data.loc[(data['age_group'] == '30-39 лет') & (data['total_income'] == 0), 'total_income'] = median_income_30
data.loc[(data['age_group'] == '30-39 лет') & (data['days_employed'] == 0), 'days_employed'] = mean_employed_30

# проверяем наличие "0" столбцах 'total_income' и 'days_employed' для возрастной группы 30-39 лет.
print (data[data['age_group'] == '30-39 лет']['total_income'].min())
print (data[data['age_group'] == '30-39 лет']['days_employed'].min())

# заменяем "0" в столбцах 'total_income' и 'days_employed' на рассчитынаю медиану и среднюю для возрастной группы 40-99 лет.
data.loc[(data['age_group'] == '40-49 лет') & (data['total_income'] == 0), 'total_income'] = median_income_40
data.loc[(data['age_group'] == '40-49 лет') & (data['days_employed'] == 0), 'days_employed'] = mean_employed_40

# проверяем наличие "0" столбцах 'total_income' и 'days_employed' для возрастной группы 30-39 лет.
print (data[data['age_group'] == '40-49 лет']['total_income'].min())
print (data[data['age_group'] == '40-49 лет']['days_employed'].min())

# заменяем "0" в столбцах 'total_income' и 'days_employed' на рассчитынаю медиану и среднюю для возрастной группы 50-59 лет.
data.loc[(data['age_group'] == '50-59 лет') & (data['total_income'] == 0), 'total_income'] = median_income_50
data.loc[(data['age_group'] == '50-59 лет') & (data['days_employed'] == 0), 'days_employed'] = mean_employed_50

# проверяем наличие "0" столбцах 'total_income' и 'days_employed' для возрастной группы 50-59 лет.
print (data[data['age_group'] == '50-59 лет']['total_income'].min())
print (data[data['age_group'] == '50-59 лет']['days_employed'].min())

#заменяем "0" в столбцах 'total_income' и 'days_employed' на рассчитынаю медиану и среднюю для возрастной группы 60+лет.
data.loc[(data['age_group'] == '60+ лет') & (data['total_income'] == 0), 'total_income'] = median_income_60
data.loc[(data['age_group'] == '60+ лет') & (data['days_employed'] == 0), 'days_employed'] = mean_employed_60

# проверяем наличие "0" столбцах 'total_income' и 'days_employed' для возрастной группы 60+ лет.
print (data[data['age_group'] == '60+ лет']['total_income'].min())
print (data[data['age_group'] == '60+ лет']['days_employed'].min())

28092.88
51
21205.28
24
25227.89
30
20667.26
58
21695.1
100


2.5.4 Посмоторим распределение клиентов по возрастным группам.

In [29]:
# Cмотрим распределение количества клиентов по возрастным группам.
data['age_group'].value_counts()

30-39 лет            5662
40-49 лет            5354
50-59 лет            4657
18-29 лет            3180
60+ лет              2500
Возраст не указан     101
Name: age_group, dtype: int64

2.5.5 Как видно из результатов сортировки по возрастным группам, основная доля людей с возрастом от 30 до 50 лет. Меньше всего людей с возрастом 60+.

2.5.6 В категории "возраст не указан" отображаются клиенты с возрастом "0".d

2.5.7 Посмотрим на таблицу после категоризации по возрастным группам и замены всех нулевых значений, а в прошлом пропусков на медиану и среднее значение.

In [30]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,недвижимость,40-49 лет
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,автомобиль,30-39 лет
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,недвижимость,30-39 лет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,образование,30-39 лет
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,свадьба,50-59 лет
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,недвижимость,18-29 лет
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,недвижимость,40-49 лет
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.93,образование,50-59 лет
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.83,свадьба,30-39 лет
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,недвижимость,40-49 лет


**Вывод**

Проведена категоризация по всей выборке клиентов по возрастным катеогриям 18-29, 30 - 39, 40 - 49, 50 - 59, 60+ лет с заменой пропущеных значений в столбцах со стажем и ежемесячным доходом. Считаю такую категоризацию удобной, позволяющей охвтатить весь возрастной диапазон клиентов и разбить на небольшие возрастные группы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим на количество клиентов в каждой категории.

In [31]:
data['children'].value_counts()

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Категоризируем наши данные следующем образом: без детей, один ребенок, многодетная семья (>= 2 ребенка). И не учитываем ошибку с 20 детьми.

In [32]:
#напишем функцию для категоризации колич-ва детей в семье
"""
если 0 детей - нет детей
если 1 ребенок - один ребенок
если 2+ детей - многодетная семья
если 20 детей - ошибка данных

"""
def children_final(child):
    if child == 0:
        return 'нет детей'
    if child == 1:
        return 'один ребенок'
    if 2 <= child <= 5:
        return 'многодетная семья'
    return 'Ошибка данных'

#применим нашу функцию к столбцу с количеством детей и сохраним результат в новом столбце children_status:

data['children_status'] = data['children'].apply(children_final)
data.head(10)
    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,children_status
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,недвижимость,40-49 лет,один ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,автомобиль,30-39 лет,один ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,недвижимость,30-39 лет,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,образование,30-39 лет,многодетная семья
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,свадьба,50-59 лет,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,недвижимость,18-29 лет,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,недвижимость,40-49 лет,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.93,образование,50-59 лет,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.83,свадьба,30-39 лет,многодетная семья
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,недвижимость,40-49 лет,нет детей


In [33]:
# создадим новую таблицу из столбцов основного датарфрейма
children_debt = data[['children_status','debt']]

# группируем по количеству детей в семье и расчитываем количество задолжности по возврату кредита в срок
children_debt_sum = data.groupby('children_status').agg({'debt':'sum'})

# группируем по количеству детей в семье и расчитываем количество кредитов
children_debt_count = data.groupby('children_status').agg({'debt':'count'})

# методом merge() собираем это все в новую таблицу children_debt_final:
children_debt_final = children_debt_sum.merge(children_debt_count, on = 'children_status', how = 'left')

# добавляем новый столбец, где расчитываем отношение кредитов с задолжность ко всем взятым кредитам по количеству детей
children_debt_final['debt_percent'] = children_debt_sum / children_debt_count

# округляем отношение до 2х знаков после запятой
children_debt_final['debt_percent'] = children_debt_final['debt_percent'].round(decimals = 2)

# переименуем значение столбцов для более понятной интерпритации методом set_axis(): 
# debts - количество кредитов с задолжностью,
#total_credits = общее количество кредитов,
#debt_percent - отношение кредитов с заолженность к общему количеству
children_debt_final.set_axis(['debts','total_credits', 'debt_percent'],axis = 'columns', inplace = True)

children_debt_final

,debts,total_credits,debt_percent
children_status,,,
Ошибка данных,8,76,0.11
многодетная семья,225,2432,0.09
нет детей,1063,14091,0.08
один ребенок,445,4855,0.09


3.1 Выше представлена таблица в которой выполнена сортировка всех клиентов из датафрейма по количеству детей в семье. 

В первом столбце 'debts' указаны все кредиты, которые имели задолженность по возврату.

Во втором столбце 'total_credits' указано общее количесво кредитов.

В третьем столбце 'debt_percent' указано отношение количества кредитов с задолженностью по возврату к общему числу кредитов.

**Вывод**

Проанализировав таблицу можно выявить, что при появлении и увелечении количества детей заметно уменьшается количество кредитов. 

У клиентов без детей количество взятых кредитов на порядок выше, чем у клиентов у которых имеются дети. Количетсво кредитов с задолжненность по возврату у таких клиентов составляет 8%.

У клиентов с одним ребенком количество взятых кредитов в 2 раза больше чем у многодетных семей, а отношение числа кредитов с задолженностью одинаковое - 9%.

Проанализировав полученные данные, можно наблюдать следующую зависимость, что общее количество кредитов с увеличением количества детей в семье уменьшается, однако отношение кредитов с задолженность к общему числу кредитов растет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим на количество клиентов в каждой категории.

In [34]:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Видно, что количество людей в разводе и вдова/вдовец имеют маленькое число клиентов, предлагаю их объединить в одну категорию вместе с "не женат/ не замужем", также категория "женат/замужем" и "гражданский брак" имеют одинакову смысловую нагрузку, эти категории тоже объединяем в одну категорию. 

In [35]:

# напишем функцию для объединения категорий по семейному положению:
"""

Если семейный статус "гражданский брак" то меняем его на "женат / замужем"
Если семейный статус "вдовец / вдова" то меняем его на "Не женат / не замужем"
Если семейный статус "в разводе" то меняем его на "Не женат / не замужем"

"""
def family_categories(status):
    
    if status == 'гражданский брак':
        return 'женат / замужем'
    elif status == 'женат / замужем':
        return 'женат / замужем'
    elif status == 'вдовец / вдова':
        return 'Не женат/ не замужем'
    elif status == 'в разводе':
        return 'Не женат/ не замужем'
    elif status == 'Не женат / не замужем':
        return 'Не женат/ не замужем'
    
# делаем копию исходного датафрейма для применения новой категоризации семейного положения на копию
data_2 = data.copy()

# применяем функцию к столбцу 'family_status'
data_2['family_status'] = data_2['family_status'].apply(family_categories)
data_2['family_status'].value_counts()

женат / замужем         16490
Не женат/ не замужем     4964
Name: family_status, dtype: int64

In [36]:
# создадим новую таблицу из столбцов основного датарфрейма
family_status_debt = data_2[['family_status','debt']]

# группируем по семейному положению и расчитываем количество задолжности по возврату кредита в срок
family_satus_debt_sum = data_2.groupby('family_status').agg({'debt':'sum'})

# группируем по семейному положению и расчитываем количество кредитов
family_status_debt_count = data_2.groupby('family_status').agg({'debt':'count'})

# методом merge() собираем это все в новую таблицу family_status_debt_final:
family_status_debt_final = family_satus_debt_sum.merge(family_status_debt_count, on = 'family_status', how = 'left')

# добавляем новый столбец, где расчитываем отношение кредитов с задолжность ко всем взятым кредитам
family_status_debt_final['debt_percent'] = family_satus_debt_sum / family_status_debt_count

# округляем отношение до 2х знаков после запятой
family_status_debt_final['debt_percent'] = family_status_debt_final['debt_percent'].round(decimals = 2)

# переименуем значение столбцов для более понятной интерпритации методом set_axis(): 
# debts - количество кредитов с задолжностью,
#total_credits = общее количество кредитов,
#debt_percent - отношение кредитов с заолженность к общему количеству
family_status_debt_final.set_axis(['debts','total_credits', 'debt_percent'],axis = 'columns', inplace = True)

# выведем таблицу по убыванию процента кредита с задолжность к общему числу кредитов
family_status_debt_final.sort_values('debt_percent', ascending = False)

,debts,total_credits,debt_percent
family_status,,,
Не женат/ не замужем,422,4964,0.09
женат / замужем,1319,16490,0.08


3.2 Выше представлена таблица в которой выполнена сортировка всех клиентов из датафрейма по семейному положению. 

В первом столбце 'debts' указаны все кредиты, которые имели задолженность по возврату.

Во втором столбце 'total_credits' указано общее количесво кредитов.

В третьем столбце 'debt_percent' указано отношение количества кредитов с задолженностью по возврату к общему числу кредитов.

**Вывод**

Самое большое количество кредитов у женатых/замужних клиентов, однако количетсво кредитов с задолженностью составляет 8%. И в абсолютном количестве это в 3 раза больше чем у клиентов без жены/мужа.

Общее количество кредитов у клиентов без жены/мужа ниже в 3 раза, однако процент отношения кредитов с задолженностью к общему числу кредитов выше на 1%.

Из таблицы можно сделать вывод, что налблюдается следующая зависимость - суммарное количество кредитов в разы превышает у клиентов с мужем/женой чем у незамужних/неженатых клиентов, и при этом имеет меньший процент кредитов с задолженностью.

Соотвественно можно сказать, что семейные пары более надеждные заемщики и основные клиенты кредитных услуг у банка.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

3.3.1 Для этого для начала выполним категоризацию клиентов датафрейма по уровню дохода.

3.3.2 Вспоминаем про 10 клиентов из датафрейма у которых не указан ни доход, ни возраст, ни трудовой стаж.

In [37]:
data[(data['total_income'] == 0) & (data['dob_years'] == 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,children_status
1890,0,0,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,0.0,недвижимость,Возраст не указан,нет детей
2284,0,0,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,0.0,недвижимость,Возраст не указан,нет детей
4062,1,0,0,среднее,1,гражданский брак,1,M,компаньон,0,0.0,недвижимость,Возраст не указан,один ребенок
5010,0,0,0,среднее,1,женат / замужем,0,F,компаньон,0,0.0,недвижимость,Возраст не указан,нет детей
6405,0,0,0,высшее,0,гражданский брак,1,F,пенсионер,0,0.0,свадьба,Возраст не указан,нет детей
6664,0,0,0,высшее,0,в разводе,3,F,пенсионер,0,0.0,недвижимость,Возраст не указан,нет детей
8565,0,0,0,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,недвижимость,Возраст не указан,нет детей
12381,3,0,0,среднее,1,женат / замужем,0,M,сотрудник,0,0.0,недвижимость,Возраст не указан,многодетная семья
13716,0,0,0,среднее,1,гражданский брак,1,F,сотрудник,0,0.0,свадьба,Возраст не указан,нет детей
19770,0,0,0,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,недвижимость,Возраст не указан,нет детей


3.3.3 Без учета этих клиентов находим минимальный и максимальный ежемесячные доходы

In [38]:
print ('Минимальный ежемесячый доход:', data[data['age_group'] != 'Возраст не указан']['total_income'].min(), 'руб.')
print ('Максимальный ежемесячый доход:', data[data['age_group'] != 'Возраст не указан']['total_income'].max(), 'руб.')

Минимальный ежемесячый доход: 20667.26 руб.
Максимальный ежемесячый доход: 2265604.03 руб.


3.3.4 Напишем функцию для категоризации по уровню дохода и добавим новый столбец в нашу таблицу с данной категорий для дальнейшего анализа клиентов.
3.3.5 Категоризацию клиентов по уровню дохода буду делать на основании статей https://www.rbc.ru/newspaper/2018/06/19/5b27bf829a794710796bbfe5 и https://www.factograph.info/a/30497993.html.
Получается что для регионов средний класс начинается при зарплате от 60+ тыс. рублей, а в Москве и Санкт-Петербурге - 110+ тыс.рублей. И заключительная категория, это средний класс в США, к которому приравнивают среднюю зарплату по миру - 220+ тыс.рублей.

In [39]:
# напишем функцию для сортировки клиентов по уровню дохода:
def income_groups(income):
    """
    Возвращает группу по уровня дохода по ежемесячному доходу:
    - "до 60 тыс.руб " при 0,1 <= income < 60000
    - "60 - 110 тыс.руб" при 60000 <= income < 110000
    - "110 - 220 тыс.руб" при 110000 <= income < 220000 
    - "220+ тыс.руб" при 220000 <= income
    - остальные 'Доход не указан'
    """
   
    if 0.1 <= income < 60000:
        return 'до 60 тыс.руб'     
          
    if 60000 <= income < 110000:
        return '60 - 110 тыс.руб'
       
    if 110000 <= income < 220000:
        return '110 - 220 тыс.руб'
       
    if 220000 <= income:
        return '220+ тыс.руб'
           
    return 'Доход не указан'

#применим нашу функцию к столбцу с 'total_income' а результаты сортировки укажем в новом столбце income_group

data['income_group'] = data['total_income'].apply(income_groups)

# проверяем результат работы функции
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,children_status,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,недвижимость,40-49 лет,один ребенок,220+ тыс.руб
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,автомобиль,30-39 лет,один ребенок,110 - 220 тыс.руб
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,недвижимость,30-39 лет,нет детей,110 - 220 тыс.руб
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,образование,30-39 лет,многодетная семья,220+ тыс.руб
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,свадьба,50-59 лет,нет детей,110 - 220 тыс.руб
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,недвижимость,18-29 лет,нет детей,220+ тыс.руб
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,недвижимость,40-49 лет,нет детей,220+ тыс.руб
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.93,образование,50-59 лет,нет детей,110 - 220 тыс.руб
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.83,свадьба,30-39 лет,многодетная семья,60 - 110 тыс.руб
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,недвижимость,40-49 лет,нет детей,110 - 220 тыс.руб


In [40]:
data['income_group'].value_counts()

110 - 220 тыс.руб    11795
60 - 110 тыс.руб      4836
220+ тыс.руб          4007
до 60 тыс.руб          806
Доход не указан         10
Name: income_group, dtype: int64

3.3.5 Теперь наши клиенты категоризированы по уровню дохода в столбце 'income_group'

Переходим к анализу таблицы и ответу на поставленный вопрос.

In [41]:
# создадим новую таблицу из столбцов основного датарфрейма
income_debt = data[['income_group','debt']]

# группируем по уровню дохода и расчитываем количество задолжности по возврату кредита в срок
income_debt_sum = data.groupby('income_group').agg({'debt':'sum'})

# группируем по цели кредита и расчитываем количество кредитов
income_debt_count = data.groupby('income_group').agg({'debt':'count'})

# методом merge() собираем это все в новую таблицу income_debt_final:
income_debt_final = income_debt_sum.merge(income_debt_count, on = 'income_group', how = 'left')

# добавляем новый столбец, где расчитываем отношение кредитов с задолжность ко всем взятым кредитам
income_debt_final['debt_percent'] = income_debt_sum / income_debt_count

# округляем отношение до 2х знаков после запятой
income_debt_final['debt_percent'] = income_debt_final['debt_percent'].round(decimals = 2)

# переименуем значение столбцов для более понятной интерпритации методом set_axis(): 
# debts - количество кредитов с задолжностью,
#total_credits = общее количество кредитов,
#debt_percent - отношение кредитов с заолженность к общему количеству
income_debt_final.set_axis(['debts','total_credits', 'debt_percent'],axis = 'columns', inplace = True)

# выведем таблицу по убыванию процента кредита с задолжность к общему числу кредитов
income_debt_final.sort_values('debt_percent', ascending = False)

,debts,total_credits,debt_percent
income_group,,,
110 - 220 тыс.руб,1002,11795,0.08
60 - 110 тыс.руб,406,4836,0.08
220+ тыс.руб,284,4007,0.07
до 60 тыс.руб,49,806,0.06
Доход не указан,0,10,0.00


3.3.6 Выше представлена таблица в которой выполнена сортировка всех клиентов из датафрейма по уровню дохода. 

В первом столбце 'debts' указаны все кредиты, которые имели задолженность по возврату.

Во втором столбце 'total_credits' указано общее количесво кредитов.

В третьем столбце 'debt_percent' указано отношение количества кредитов с задолженностью по возврату к общему числу кредитов.

**Вывод**

Самое большое количество кредитов у клиентов с доходом от 110 до 220 тыс. рублей, составляет половину всех клиентов датафрейма и у них же самый высокий процент отношения кредитов с задолженостью к общему числу кредитов.

Самое малеьнкое количество кредитов у клиентов с доходом до 60 тыс. рублей, а так же самый низкий процент отношения кредитов с задолженость к общему числу кредитой.

Наблюдается следующая закономерность, что основная площадь нормального распределния по числу кредитов ложится на клиентов с доходом от 110 до 220 тыс. рублей. Хвостами распределния являются клиенты с доходами от 60 до 110 тыс. рублей и 220+ тыс. рублей. Однако стоит заметить, что число кредитов с задолженностью падает с ростом уровня дохода - 0.07 у клиентов с доходом больше 220 тыс. рублей. 

Однако у клиентов с доходом 60 тыс. рублей тоже маленький процент отоншения кредитов с задолженностью к общему числу кредитов. Возможно этот момент связан с количеством кредитов в целом для людей с таким уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
# создадим новую таблицу из столбцов основного датарфрейма
purpose_debt = data[['purpose','debt']]

# группируем по цели крдита и расчитываем количество задолжности по возврату кредита в срок
purpose_debt_sum = data.groupby('purpose').agg({'debt':'sum'})

# группируем по цели кредита и расчитываем количество кредитов
purpose_debt_count = data.groupby('purpose').agg({'debt':'count'})

# методом merge() собираем это все в новую таблицу purpose_debt_final:
purpose_debt_final = purpose_debt_sum.merge(purpose_debt_count, on = 'purpose', how = 'left')

# добавляем новый столбец, где расчитываем отношение кредитов с задолжность ко всем взятым кредитам
purpose_debt_final['debt_percent'] = purpose_debt_sum / purpose_debt_count

# округляем отношение до 2х знаков после запятой
purpose_debt_final['debt_percent'] = purpose_debt_final['debt_percent'].round(decimals = 2)

# переименуем значение столбцов для более понятной интерпритации методом set_axis(): 
# debts - количество кредитов с задолжностью,
#total_credits = общее количество кредитов,
#debt_percent - отношение кредитов с заолженность к общему количеству
purpose_debt_final.set_axis(['debts','total_credits', 'debt_percent'],axis = 'columns', inplace = True)

# выведем таблицу по убыванию процента кредита с задолжность к общему числу кредитов
purpose_debt_final.sort_values('debt_percent', ascending = False)

,debts,total_credits,debt_percent
purpose,,,
автомобиль,403,4306,0.09
образование,370,4013,0.09
свадьба,186,2324,0.08
недвижимость,782,10811,0.07


3.4 Выше представлена таблица в которой выполнена сортировка всех клиентов из датафрейма по цели кредита. 

В первом столбце 'debts' указаны все кредиты, которые имели задолженность по возврату.

Во втором столбце 'total_credits' указано общее количесво кредитов.

В третьем столбце 'debt_percent' указано отношение количества кредитов с задолженностью по возврату к общему числу кредитов.

**Вывод**

Самое большое количество кредитов для недвижимости. У них же самый маленький процент отношения кредитов с задолженность к общему числу кредитов.

Самый большое отношение кредитов с задолженность к общему числу кредитов - автомобильный кредит и кредит на образование.

Самое маленькое количесвто кредитов - на свадьбу. Отошнение невозвратных к общему числу - 8%, но в абсолютных значениях - самый маленький.

Из таблицы можно сделать вывод, что долгосрочные кредиты для недвижимости - самые популярные и имеют самый маленький процент отношения кредитов с заолженность к общему числу кредитов. Несмотря на то, что кредиты на недвижмость самые долгосрочные, количество кредитов с задолженность у них самый малельнкий.

А вот кредиты на автомобиль и учебу, которые имееют меньшие суммы и сроки выплаты имеют отношение кредитов с заолженность к общему числу - самое высокое.

Свадебные кредиты - самые непопулярные.

## Шаг 4. Общий вывод

4.1 Выполнен анализ датафрейма с клиентами банка для оценки влияния семейного положения и количества детей на факт погашения кредита в срок.

4.2 Из полученных данных можно выделить, что 3/4 кредитов  из датафрейма приходится на семейную пару. А 1/4 на клиентов без пары.

4.3. Самый популярная цель для кредита - недвижимость (10814). На втором и третьем месте - автомобиль (4308) и образование (4014). Кредит для свадьбы - самый непопулярный (2335).

4.4 С увелечением количества детей в семье падает общее количество кредитов, и возрастает отношение кредитов с задолженность к общему их количеству.

4.5 Основное число кредитов в зависимости от дохода клиентов напоминает нормальное распределение, где основная масса и пик этого распределния сосредоточен у клиентов с доходом от 110 до 220 тыс. рублей. Хвотсы этого распределния - клиенты с доходом больше 220 тыс. рублей и от 60 до 110 тыс. рублей. Клиенты с доходом меньше 60 тыс. рублей не охотно берут кредиты, но если берут, то отношение кредитов с задолженность к общему числу - 6%.

Из результатов получается, что основная аудитория клиентов банка для кредитов это семьи без детей или с одним ребенком. С ежемесячным доходом в диапазоне от 110 до 220 тыс. рублей("средний" класс). 

Самый популярчный кредит для них будет недвижимость, далее автомобоиль или образование. Однако стоит отметить, что вместе с большим количеством кредитов, довольно высокий процент отношения кредитов с задолженностью к общему количеству, особенно на автомоболи и образование.

Из этого можно сделать вывод, что молодая семья без детей или с 1 ребенком с целью взятие кредита на недвижимость самый надеждный заемщик.

